In [1]:
#NOAA Weather data download for Portland, Oregon

import requests
import pandas as pd

In [ ]:
NOAA_TOKEN = 'XX'

In [7]:
from datetime import datetime, timedelta

#NOAA API Headers
headers = {'token': NOAA_TOKEN}

#Portland Airport Station ID (GHCND:USW00024229)
station_id = 'GHCND:USW00024229'

#Date range
start_date = '2025-01-01'
end_date = '2025-05-31'

datatype_ids = ['STATION_NAME','STATION','LONGITUDE','LATITUDE','ELEVATION','TMIN', 'TMAX', 'PRCP']

""" def daterange(start_date, end_date, delta_days = 10):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    while start < end:
        chunk_end = min(start + timedelta(days = delta_days - 1), end)
        yield start.strftime("%Y-%m-%d"), chunk_end.strftime("%Y-%m-%d")
        start = chunk_end + timedelta(days=1) """

def fetch_noaa_data(station_id, datatype_ids, start_date,
                    end_date, token):
    results = []

    for dtype in datatype_ids:
        print(f"Fetching {dtype} data...")
        #for s, e in daterange(start_date, end_date):
        params = {
            'datasetid': 'GHCND',
            'stationid': station_id,
            'datatypeid': dtype,
            'startdate': start_date,
            'enddate': end_date,
            'units': 'metric',
            'limit': 1000
        }

        response = requests.get(
            'https://www.ncdc.noaa.gov/cdo-web/api/v2/data',
            headers={'token': token},
            params=params            
        )

        if response.status_code == 200:
            data = response.json().get('results', [])
            results.extend(data)
        else:
            print(f"Error fetching {dtype}: {response.status_code} - {response.text}")
        
    return pd.DataFrame(results)

raw_df = fetch_noaa_data(station_id, datatype_ids, start_date, end_date, NOAA_TOKEN)

if not raw_df.empty:
    raw_df['date'] = pd.to_datetime(raw_df['date']).dt.date

    df = raw_df.pivot_table(index='date', columns=
                            'datatype')
    
    if 'MIN' in df.columns: df['TMIN'] = df['TMIN']/10
    if 'MAX' in df.columns: df['TMAX'] = df['TMIN']/10
    if 'PRCP' in df.columns: df['PRCP'] = df['PRCP']/10

    print(df.head())


else:
    print("No data available")



Fetching STATION_NAME data...
Fetching STATION data...
Fetching LONGITUDE data...
Fetching LATITUDE data...
Fetching ELEVATION data...
Fetching TMIN data...
Fetching TMAX data...
Fetching PRCP data...
           value           
datatype    PRCP  TMAX TMIN
date                       
2025-01-01   0.3   6.7  4.4
2025-01-02  12.2   6.7  5.0
2025-01-03  15.7   8.3  6.1
2025-01-04  15.2  10.6  6.1
2025-01-05   2.0  12.8  8.9


C:\Users\brosb\AppData\Local\Temp\ipykernel_13140\1927588234.py:60: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df = raw_df.pivot_table(index='date', columns=


In [8]:
df.head()

value           
datatype    PRCP  TMAX TMIN
date                       
2025-01-01   0.3   6.7  4.4
2025-01-02  12.2   6.7  5.0
2025-01-03  15.7   8.3  6.1
2025-01-04  15.2  10.6  6.1
2025-01-05   2.0  12.8  8.9

In [9]:
#get metadata about the station
station_metadata = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/stations/GHCND:USW00024229',
                                headers={'token': NOAA_TOKEN}).json()

print("Station info:")
print('Name:', station_metadata.get('name'))
print('Latitude:', station_metadata.get('latitude'))
print('Longitude:', station_metadata.get('longitude'))
print('Elevation:', station_metadata.get('elevation'))

Station info:
Name: PORTLAND INTERNATIONAL AIRPORT, OR US
Latitude: 45.59578
Longitude: -122.60919
Elevation: 6.7


In [10]:
#Add station metadata to each row
for key, value in station_metadata.items():
    df[key] = value

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 151 entries, 2025-01-01 to 2025-05-31
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (value, PRCP)      151 non-null    float64
 1   (value, TMAX)      151 non-null    float64
 2   (value, TMIN)      151 non-null    float64
 3   (elevation, )      151 non-null    float64
 4   (mindate, )        151 non-null    object 
 5   (maxdate, )        151 non-null    object 
 6   (latitude, )       151 non-null    float64
 7   (name, )           151 non-null    object 
 8   (datacoverage, )   151 non-null    int64  
 9   (id, )             151 non-null    object 
 10  (elevationUnit, )  151 non-null    object 
 11  (longitude, )      151 non-null    float64
dtypes: float64(6), int64(1), object(5)
memory usage: 15.3+ KB


In [12]:
df.head()

value            elevation     mindate     maxdate  latitude  \
datatype    PRCP  TMAX TMIN                                               
date                                                                      
2025-01-01   0.3   6.7  4.4       6.7  1936-05-01  2025-06-05  45.59578   
2025-01-02  12.2   6.7  5.0       6.7  1936-05-01  2025-06-05  45.59578   
2025-01-03  15.7   8.3  6.1       6.7  1936-05-01  2025-06-05  45.59578   
2025-01-04  15.2  10.6  6.1       6.7  1936-05-01  2025-06-05  45.59578   
2025-01-05   2.0  12.8  8.9       6.7  1936-05-01  2025-06-05  45.59578   

                                             name datacoverage  \
datatype                                                         
date                                                             
2025-01-01  PORTLAND INTERNATIONAL AIRPORT, OR US            1   
2025-01-02  PORTLAND INTERNATIONAL AIRPORT, OR US            1   
2025-01-03  PORTLAND INTERNATIONAL AIRPORT, OR US            1   
2025-01-04  PORTLAND INTERNATIONAL AIRPORT, OR US            1   
2025-01-05  PORTLAND INTERNATIONAL AIRPORT, OR US            1   

                           id elevationUnit  longitude  
datatype                                                
date                                                    
2025-01-01  GHCND:USW00024229        METERS -122.60919  
2025-01-02  GHCND:USW00024229        METERS -122.60919  
2025-01-03  GHCND:USW00024229        METERS -122.60919  
2025-01-04  GHCND:USW00024229        METERS -122.60919  
2025-01-05  GHCND:USW00024229        METERS -122.60919

In [13]:
df.to_csv('Portland_Weather_input.csv', index=False)